# Process ADB MRIO Excel files

This notebook converts the ADB MRIO Excel files into machine-readable format and saves them as space-efficient Parquet files. Results are stored in `data/mrio/`. 

In [21]:
import pandas as pd
import os
import re
import duckdb

In [11]:
def process_table(df):
    '''
    Reformat the raw Excel table to make it machine-readable
    '''
    
    # Remove the last row and the first 2 columns
    df = df.drop(df.index[-1])
    df = df.iloc[:, 2:]

    # Collapse MultiIndex headers into one
    df.columns = [f'{level_1}_{level_2}' for level_1, level_2 in df.columns]

    # Rename the ToT column
    colnames = df.columns.tolist()
    mapping = {colnames[-1]: 'ToT'}
    df = df.rename(columns=mapping)

    # Fix row labels
    rowlabels = [f"{c}_{d}" if not (pd.isna(c) or c == 'ToT') else d for c, d in zip(df.iloc[:, 0], df.iloc[:, 1])]
    df.insert(2, 'si', rowlabels)
    df = df.iloc[:, 2:]
    
    # Drop intermediates totals
    df = df.drop(df[df['si'] == 'r60'].index)

    # Replace blank cells with zero
    df = df.replace(' ', 0)

    return df

In [15]:
def load_and_save(inputfolder, outputfile, version=None):
    '''
    Load raw Excel tables, process them using process_table(), stack all years into one file, 
    then export as Parquet file
    '''

    mrio = pd.DataFrame()
    
    filelist = [file for file in os.listdir(f'../data/raw/{inputfolder}') if not file.startswith('.')]
    filelist.sort()

    for file in filelist:
        
        year = re.search('[0-9]{4}', file).group()
        
        mrio_t = pd.read_excel(
            f'../data/raw/{folder}/{file}',
            skiprows=5,
            header=[0,1]
        )
        
        mrio_t = process_table(mrio_t)
        mrio_t.insert(0, 't', year)
        mrio = pd.concat([mrio, mrio_t], ignore_index=True)

        print(f'{year} done')
        
    if version is None:
        mrio.to_parquet(f'../data/mrio/{outputfile}.parquet', index=False)
    else:
        mrio.to_parquet(f'../data/mrio/{outputfile}_{version}.parquet', index=False)

## Quick view

The raw Excel table looks like this:

In [19]:
folder = 'ADB MRIO, 72 economies'
file = 'ADB-MRIO-2022_June2023.xlsx'
pd.read_excel(f'../data/raw/{folder}/{file}', skiprows=5, header=[0,1])

Unnamed: 0_level_0                                 Unnamed: 1_level_0  \
     Unnamed: 0_level_1                                 Unnamed: 1_level_1   
0                   NaN        Agriculture, hunting, forestry, and fishing   
1                   NaN                               Mining and quarrying   
2                   NaN                       Food, beverages, and tobacco   
3                   NaN                      Textiles and textile products   
4                   NaN            Leather, leather products, and footwear   
...                 ...                                                ...   
2559                NaN  Purchases on the domestic territory by non-res...   
2560                NaN                        Value added at basic prices   
2561                NaN                    International Transport Margins   
2562                NaN                                              TOTAL   
2563                NaN  $ = United States dollar; cif = cost, insuranc...   

     Unnamed: 2_level_0 Unnamed: 3_level_0            AUS                 \
     Unnamed: 2_level_1 Unnamed: 3_level_1             c1             c2   
0                   AUS                 c1   29942.469136     415.630105   
1                   AUS                 c2     408.466823   23478.652369   
2                   AUS                 c3    2142.881899     436.996636   
3                   AUS                 c4      29.911135      58.999450   
4                   AUS                 c5       2.953425       5.532104   
...                 ...                ...            ...            ...   
2559                ToT                r63       0.000000       0.000000   
2560                ToT                r64   50885.109816  218767.978030   
2561                ToT                trs       0.000000       0.000000   
2562                ToT                r69  127271.626387  343094.005312   
2563                NaN                NaN            NaN            NaN   

                                                           ...           NZL  \
                c3           c4          c5            c6  ...            F2   
0     37563.566708   699.332810   54.983390   3031.321682  ...  3.438057e-05   
1       490.870782    11.665751    1.581656     69.615110  ...  8.149034e-09   
2      6698.080986     7.309882  204.879284     15.151942  ...  1.570918e-01   
3        23.528795    67.287440    3.730287      5.836821  ...  2.065301e-03   
4         2.360051    14.889115   10.812711      0.265745  ...  1.251363e-02   
...            ...          ...         ...           ...  ...           ...   
2559      0.000000     0.000000    0.000000      0.000000  ...  0.000000e+00   
2560  19387.934022  1447.780988  332.052873   3514.067495  ...  0.000000e+00   
2561      0.000000     0.000000    0.000000      0.000000  ...  0.000000e+00   
2562  82512.185782  3559.332836  935.852043  10816.516349  ...  2.506376e+03   
2563           NaN          NaN         NaN           NaN  ...           NaN   

                                                        RoW               \
                F3            F4           F5            F1           F2   
0         0.131504      0.087051    -0.618808  1.059757e+00     0.000725   
1         0.009656      0.000000     0.253144  2.602228e+01     0.011817   
2         1.126099      0.000000  -155.653489  9.750351e+02     0.695487   
3         0.000729      1.611411    -0.094581  2.003621e+01     0.002377   
4         0.469973      0.000000     0.361366  1.015215e+02     0.009110   
...            ...           ...          ...           ...          ...   
2559      0.000000      0.000000     0.000000  0.000000e+00     0.000000   
2560      0.000000      0.000000     0.000000  0.000000e+00     0.000000   
2561      0.000000      0.000000     0.000000  0.000000e+00     0.000000   
2562  51009.288711  62165.886443  1576.861705  5.391299e+06  3758.938281   
2563           NaN           NaN          NaN   

The goal of this notebook is to transform it into this:

In [22]:
duckdb.sql("SELECT * FROM read_parquet('../data/mrio/adb-mrio.parquet')").df()

,t,si,AUS_c1,AUS_c2,AUS_c3,AUS_c4,AUS_c5,AUS_c6,AUS_c7,AUS_c8,...,NZL_F2,NZL_F3,NZL_F4,NZL_F5,RoW_F1,RoW_F2,RoW_F3,RoW_F4,RoW_F5,ToT
0,2017,AUS_c1,12199.559183,214.280421,22912.845631,634.534126,54.032134,1754.892087,19.475614,19.776085,...,0.000000,0.306421,4.398006,-2.632899,7.553193e+02,0.000078,5.577177,9.456021e+00,0.021587,7.659055e+04
1,2017,AUS_c2,250.316692,16671.934616,481.699583,9.676875,1.191425,44.117673,291.621069,1286.044207,...,0.000000,0.000000,11.431150,21.523687,6.704805e+00,0.001411,0.686634,2.823491e+01,0.003934,2.199046e+05
2,2017,AUS_c3,1088.256793,320.590655,6280.283519,5.488313,162.329879,16.741351,36.602804,42.586943,...,0.000000,0.000000,0.000000,15.801672,7.641520e+02,0.019817,9.609331,1.339063e-04,0.132746,6.887613e+04
3,2017,AUS_c4,18.164412,36.927218,21.299460,25.558736,1.896831,6.166664,19.265706,1.765148,...,0.000000,0.000000,1.204813,-0.370723,1.553536e+02,0.004976,1.319984,2.417384e-03,-0.012125,3.577509e+03
4,2017,AUS_c5,3.681203,6.056107,7.035795,24.693916,15.141007,0.715095,12.437081,0.353283,...,0.000000,0.000000,0.000000,-0.542867,2.374779e+01,0.000040,0.246063,5.799603e-02,0.829850,8.304065e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15367,2022,r62,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,2.562686e+05
15368,2022,r63,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,-2.238295e+05
15369,2022,r64,50885.109816,218767.978030,19387.934022,1447.780988,332.052873,3514.067495,8333.441488,2409.686996,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,9.652560e+07
15370,2022,trs,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00


## ADB MRIO 72 economies

In [16]:
folder = 'ADB MRIO, 72 economies'
load_and_save(inputfolder=folder, outputfile='adb-mrio')

2017 done
2018 done
2019 done
2020 done
2021 done
2022 done


## ADB MRIO 62 economies

In [54]:
folder = 'ADB MRIO, 62 economies'
load_and_save(inputfolder=folder, outputfile='adb-mrio62')

2008 done
2009 done
2011 done
2018 done
2016 done
2019 done
2010 done
2012 done
2015 done
2014 done
2013 done
2017 done
2007 done
2021 done
2020 done
2000 done


## ADB MRIO 72 economies (version Jun 2023)

In [55]:
folder = '9 MRIO 2020-2022 for upload (ao Jun 2023)'
load_and_save(inputfolder=folder, outputfile='adb-mrio', version='jun2023')

2021 done
2022 done
2020 done
